In [4]:
pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 12.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 56.3 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 3.3 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 907.9/907.9 kB 7.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.3/278.3 kB 25.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install --upgrade langchain

You should consider upgrading via the '/Users/cauhike/Documents/GitHub/rag/env/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


## Ollama (gemma)

In [3]:
import chromadb
# chroma_client = chromadb.Client()
chroma_client = chromadb.HttpClient(host="localhost", port=8000)
print(chroma_client.list_collections())

[Collection(name=pdf_test_240531), Collection(name=gemma_test_240501), Collection(name=address_business_guide_openai), Collection(name=address_business_guide), Collection(name=public_doc_address_spliter_240503), Collection(name=public_doc_address_240503), Collection(name=address_business_guide_llama3), Collection(name=llama3_test_240501)]


In [2]:
collection = chroma_client.create_collection(name="address_business_guide")
collection.peek()

{'ids': [],
 'embeddings': [],
 'metadatas': [],
 'documents': [],
 'data': None,
 'uris': None}

In [17]:
from langchain_community.document_loaders import PyMuPDFLoader
loader = PyMuPDFLoader("data/2021_주소정보_업무편람.pdf")
pages = loader.load()
len(pages)

560

In [6]:
pages

[Document(page_content=' \n', metadata={'source': 'data/2021_주소정보_업무편람.pdf', 'file_path': 'data/2021_주소정보_업무편람.pdf', 'page': 0, 'total_pages': 560, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'macOS 버전 14.2.1(빌드 23C71) Quartz PDFContext, AppendMode 1.1', 'creationDate': "D:20230703011412Z00'00'", 'modDate': "D:20240215090249Z00'00'", 'trapped': ''}),
 Document(page_content='', metadata={'source': 'data/2021_주소정보_업무편람.pdf', 'file_path': 'data/2021_주소정보_업무편람.pdf', 'page': 1, 'total_pages': 560, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'macOS 버전 14.2.1(빌드 23C71) Quartz PDFContext, AppendMode 1.1', 'creationDate': "D:20230703011412Z00'00'", 'modDate': "D:20240215090249Z00'00'", 'trapped': ''}),
 Document(page_content='i\n주소정보 업무편람을 발간하면서\n새로운 주소체계를도입하여 정착발전시키는 중차대한 임무를 훌륭히 수행해온 주소정보 \n \n·\n업무 담당 공무원 여러분! \n도로명주소가 국민 생활 속에 안착되고 있는 것은 여러분께서 도로명주소를 부여하고 안내\n시설을 설치하기 

In [9]:
import chromadb
import uuid
from tqdm import tqdm
from langchain.text_splitter import RecursiveCharacterTextSplitter
from chromadb.utils import embedding_functions
from chromadb.utils.embedding_functions import OllamaEmbeddingFunction

def insert(content):
    # client = chromadb.Client()
    client = chromadb.HttpClient(host="localhost", port=8000)
    # openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    #             model_name="text-embedding-ada-002"
    #         )
    ollama_ef = OllamaEmbeddingFunction(url="http://localhost:11434/api/embeddings", model_name="gemma")
    collection = client.get_collection(name="address_business_guide", embedding_function=ollama_ef)

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)
    docs = text_splitter.split_documents(content)

    for doc in tqdm(docs):
        uuid_val = uuid.uuid1()
        # print("Inserted documents for ", uuid_val)
        collection.add(ids=[str(uuid_val)], documents=doc.page_content)

In [10]:
insert(pages)

100%|██████████| 2383/2383 [18:26<00:00,  2.15it/s]


18분 걸림 대박!

In [23]:
import chromadb
from langchain.vectorstores import Chroma
# from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings

def queryDB(query):
    client = chromadb.HttpClient(host="localhost", port=8000)
    # embedding_function = OpenAIEmbeddings()
    embedding_function = OllamaEmbeddings(model="gemma")
    db4 = Chroma(client=client, collection_name="address_business_guide", embedding_function=embedding_function)
    docs = db4.similarity_search_with_score(query) ## 유사도 점수까지 확인: https://wikidocs.net/234094
    # retriever = db4.as_retriever(search_type="mmr") ## mmr 검색
    
    return docs # retriever.get_relevant_documents(query)[0]

In [24]:
query="사물주소에 대해서 설명해줘"
result = queryDB(query)
result

[(Document(page_content='시군구의 주소정보 전산체계 관리책임자는 보안관리를 위하여 관계공\n·\n, \n·\n·\n○ \n무원을 보안담당자로 지정하고 주기적으로 보안점검을 실시하고 미흡한 부분\n을 보완하여야 함'),
  30802.814453125),
 (Document(page_content='- \n교량은 도로의 실폭이 보이지 않도록 작성\n- \n입체교차로는 고가도로 작성시 고가부분을 연결되게 보이도록 작성\n- \n지하도는 도로의 실폭이 보이고 지하로 경계가 지하도 실폭과 지하도의 경\n- \n계가 함께 보이도록 작성\n평면교차로는 교차로로 작성하되 교차하는 도로의 실폭 간에 연결되게 보이\n- \n도록 작성'),
  31639.794921875),
 (Document(page_content='다\n97\n29530\n다가구주택의 경우 주민등록상의 주소의 기재는 지번만 기재하는 것으\n로 충분하며주민등록 전입신고 시 법령상 공동주택의 경우와 달리 동\n, \n, \n호수를 기재하여야 할 근거가 없음'),
  31725.5703125),
 (Document(page_content='시장등은 지주등의 본래 용도에 지장을 주지 아니하는 범위에서 도로명판 및 \n○ \n기초번호판을 설치하는데 지주등을 사용할 수 있음법 \n(\n§9\n)\n③④⑤\n지주등을 사용하려면 미리 그 지주등의 설치자 또는 관리자와 협의하여야 하\n- \n며\n협의 요청을 받은 자는 특별한 사유가 없으면 지주등의 사용에 협조하여\n, \n야 함'),
  32128.287109375)]

In [25]:
max_similarity_doc = min(result, key=lambda x: x[1])[0].page_content
max_similarity_doc

'시군구의 주소정보 전산체계 관리책임자는 보안관리를 위하여 관계공\n·\n, \n·\n·\n○ \n무원을 보안담당자로 지정하고 주기적으로 보안점검을 실시하고 미흡한 부분\n을 보완하여야 함'

In [26]:
import ollama

output = ollama.generate(
    model="gemma",
    prompt=f"Using the data: {max_similarity_doc}. Respond to this prompt: {query}. If data doesn't have an appropriate value to answer the question in the prompt, answer 'I can't answer the question.' 한국어로 대답해줘 "
)

print(output['response'])

제공된 데이터에는 사물주소에 대한 정보가 포함되어 있지 않습니다. 따라서 이 질문에 답할 수 없습니다.


## OpenAI API 
* 어느 정도 나오기는 한데, retrieval한 결과를 기반으로 답변하는 것 같지가 않음. chunk size를 바꿔서 테스트 해볼 필요 있음

In [48]:
pip install langchain_core

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
os.environ.get("OPENAI_API_KEY")

In [5]:
collection = chroma_client.get_or_create_collection(name="address_business_guide_openai")
collection.peek()

{'ids': ['00086fa6-230a-11ef-80fa-ea9d1e35f997',
  '00277652-230b-11ef-80fa-ea9d1e35f997',
  '00357096-230a-11ef-80fa-ea9d1e35f997',
  '00524b48-230b-11ef-80fa-ea9d1e35f997',
  '005c01f2-230a-11ef-80fa-ea9d1e35f997',
  '0077c198-230b-11ef-80fa-ea9d1e35f997',
  '00898c44-230a-11ef-80fa-ea9d1e35f997',
  '00a258ea-230b-11ef-80fa-ea9d1e35f997',
  '00b8522c-230a-11ef-80fa-ea9d1e35f997',
  '00cd7d5e-230b-11ef-80fa-ea9d1e35f997'],
 'embeddings': [[-0.010541114024817944,
   0.0034577432088553905,
   0.0063524795696139336,
   -0.03144697844982147,
   -0.022466003894805908,
   0.029819006100296974,
   -0.022059010341763496,
   -0.004551536403596401,
   -0.011327967047691345,
   0.01773132011294365,
   -9.49649911490269e-05,
   -0.0002229981473647058,
   -0.03169117495417595,
   0.0042971656657755375,
   -0.006837479304522276,
   0.0008618920692242682,
   0.007739646825939417,
   -0.014109084382653236,
   0.0025403134059160948,
   -0.018423208966851234,
   0.016184747219085693,
   -0.004480312578

In [31]:
import chromadb
import uuid
from tqdm import tqdm
from langchain.text_splitter import RecursiveCharacterTextSplitter
from chromadb.utils import embedding_functions
from chromadb.utils.embedding_functions import OllamaEmbeddingFunction

def insert(content):
    # client = chromadb.Client()
    client = chromadb.HttpClient(host="localhost", port=8000)
    openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                model_name="text-embedding-ada-002"
            )
    # ollama_ef = OllamaEmbeddingFunction(url="http://localhost:11434/api/embeddings", model_name="gemma")
    collection = client.get_collection(name="address_business_guide_openai", embedding_function=openai_ef)

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)
    docs = text_splitter.split_documents(content)

    for doc in tqdm(docs):
        uuid_val = uuid.uuid1()
        # print("Inserted documents for ", uuid_val)
        collection.add(ids=[str(uuid_val)], documents=doc.page_content)

In [32]:
insert(pages)

100%|██████████| 2383/2383 [12:01<00:00,  3.30it/s]


- 시간: 12분
- 차감 금액: 0.06달러

In [1]:
import chromadb
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
# from langchain_community.embeddings import OllamaEmbeddings

def queryDB(query):
    client = chromadb.HttpClient(host="localhost", port=8000)
    embedding_function = OpenAIEmbeddings()
    # embedding_function = OllamaEmbeddings(model="gemma")
    db4 = Chroma(client=client, collection_name="address_business_guide_openai", embedding_function=embedding_function)
    docs = db4.similarity_search_with_score(query) ## 유사도 점수까지 확인: https://wikidocs.net/234094
    # retriever = db4.as_retriever(search_type="mmr") ## mmr 검색
    
    return docs # retriever.get_relevant_documents(query)[0]

/Users/cauhike/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
query="사물주소에 대해서 설명해줘"
result = queryDB(query)
result

In [10]:
max_similarity_doc = min(result, key=lambda x: x[1])[0].page_content
max_similarity_doc

'대체주소는 도로명주소가 부여된 건물등의 별도의 출입구 또는 사물주소가 부여된 \n○ \n시설물의 별도의 기준점에 부여된 주소정보를 말함\n2) 별칭\n○ 참조체계에 따라 작성된 주소정보의 구성요소가 아니라 상호\n건물명\n시설물명 \n, \n, \n등과 같이 해당 객체를 부를 때 사용되는 여러 명칭을 말함\n3) 시간정보'

In [13]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# 프롬프트 생성
prompt = f"""Answer the question based only on the following context, which can include texts:
{max_similarity_doc}
If you can't find the answer in the context, answer "I can't answer the question.".
Please answer in Korean.
"""

# LLM
model = ChatOpenAI(temperature=0, model="gpt-4o")

# RAG pipeline
chain = (
    RunnablePassthrough(lambda x: prompt)  # 함수를 직접 전달
    | model
    | StrOutputParser()
)

# 파이프라인 실행
output = chain.invoke(query)
print(output)


사물주소는 사물인터넷(IoT) 기술을 활용하여 물리적인 사물에 고유한 디지털 주소를 부여하는 개념입니다. 이를 통해 사물들은 인터넷을 통해 서로 소통하고 데이터를 주고받을 수 있습니다. 사물주소는 주로 다음과 같은 요소들로 구성됩니다:

1. **고유 식별자(UID)**: 각 사물에 부여되는 고유한 식별 번호입니다. 이 번호를 통해 특정 사물을 다른 사물과 구분할 수 있습니다.

2. **네트워크 주소**: 사물이 인터넷에 연결될 수 있도록 IP 주소나 MAC 주소와 같은 네트워크 주소가 필요합니다.

3. **메타데이터**: 사물의 상태, 위치, 기능 등을 설명하는 추가 정보입니다. 예를 들어, 온도 센서라면 현재 온도, 위치, 배터리 상태 등의 정보가 포함될 수 있습니다.

사물주소는 다양한 응용 분야에서 활용될 수 있습니다. 예를 들어, 스마트 홈에서는 가전제품들이 서로 소통하여 효율적으로 에너지를 관리할 수 있고, 스마트 시티에서는 교통 신호등, 가로등, 쓰레기통 등이 서로 연결되어 도시의 효율성을 높일 수 있습니다.

사물주소의 주요 장점은 다음과 같습니다:

- **효율성 향상**: 사물들이 서로 소통함으로써 자원을 효율적으로 사용할 수 있습니다.
- **자동화**: 많은 작업이 자동화되어 인간의 개입이 줄어듭니다.
- **데이터 수집 및 분석**: 다양한 사물에서 데이터를 수집하여 분석함으로써 더 나은 의사결정을 할 수 있습니다.

하지만 사물주소를 구현하는 데에는 보안 문제, 표준화 문제, 데이터 프라이버시 문제 등 여러 도전 과제도 존재합니다. 이러한 문제들을 해결하기 위해 다양한 연구와 기술 개발이 진행되고 있습니다.


In [15]:
## 한번에 실행하는 코드

from langchain_openai import ChatOpenAI

query="사물주소 부여 원칙에 대해서 설명해줘"
result = queryDB(query)
max_similarity_doc = min(result, key=lambda x: x[1])[0].page_content


# 프롬프트 생성
prompt = f"""Answer the question based only on the following context, which can include texts:
{max_similarity_doc}
If you can't find the answer in the context, answer "I can't answer the question.".
Please answer in Korean.
"""

# LLM
model = ChatOpenAI(temperature=0, model="gpt-4o")

# RAG pipeline
chain = (
    RunnablePassthrough(lambda x: prompt)  # 함수를 직접 전달
    | mod
    | StrOutputParser()
)

# 파이프라인 실행
output = chain.invoke(query)
print(max_similarity_doc)
print('========================')
print(output)


제
장 주소정보 부여변경폐지
III
·
·
 ■  ■  ■
243
2
사물주소의 부여변경폐지 기준
·
·
가
사물주소의 부여변경폐지 기준
. 
·
·
사물주소 부여 원칙영 
1) 
(
§41
)
①
행정안전부장관이 고시한 사물주소 부여 대상 시설물시설 및 장소마다  하나의 
(
) 
○ 
사물번호 부여
사물주소 부여 원칙은 사물인터넷(IoT) 환경에서 각 사물에 고유한 주소를 부여하여 네트워크 상에서 식별하고 통신할 수 있도록 하는 원칙을 의미합니다. 이 원칙은 사물인터넷의 효율적 운영과 관리에 필수적입니다. 주요 원칙은 다음과 같습니다:

1. **고유성 (Uniqueness)**:
   - 각 사물에 부여되는 주소는 전 세계적으로 유일해야 합니다. 이는 충돌을 방지하고, 각 사물을 명확하게 식별할 수 있도록 합니다.

2. **확장성 (Scalability)**:
   - 사물의 수가 기하급수적으로 증가할 것을 대비하여, 주소 체계는 확장 가능해야 합니다. IPv6와 같은 주소 체계는 이러한 확장성을 제공합니다.

3. **유연성 (Flexibility)**:
   - 다양한 종류의 사물과 다양한 네트워크 환경에서 사용될 수 있도록 주소 체계는 유연해야 합니다. 이는 다양한 프로토콜과의 호환성을 의미합니다.

4. **관리 용이성 (Manageability)**:
   - 주소 체계는 관리가 용이해야 합니다. 이는 주소의 할당, 변경, 해제 등이 효율적으로 이루어질 수 있어야 함을 의미합니다.

5. **보안성 (Security)**:
   - 사물 주소는 보안성을 갖추어야 합니다. 이는 주소의 위조나 변조를 방지하고, 사물 간의 안전한 통신을 보장하기 위함입니다.

6. **표준화 (Standardization)**:
   - 주소 체계는 국제적으로 표준화되어야 합니다. 이는 다양한 제조업체와 서비스 제공자가 동일한 기준을 따를 수 있도록 하여 상호 운용성을 보장합니다.

7. **지속 가능성 (Sustainability)**:
   - 주소 체계는 장기

## Ollama(llama3:70b)

In [16]:
collection = chroma_client.create_collection(name="address_business_guide_llama3")
collection.peek()

{'ids': [],
 'embeddings': [],
 'metadatas': [],
 'documents': [],
 'data': None,
 'uris': None}

In [18]:
import chromadb
import uuid
from tqdm import tqdm
from langchain.text_splitter import RecursiveCharacterTextSplitter
from chromadb.utils import embedding_functions
from chromadb.utils.embedding_functions import OllamaEmbeddingFunction

def insert(content):
    # client = chromadb.Client()
    client = chromadb.HttpClient(host="localhost", port=8000)
    # openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    #             model_name="text-embedding-ada-002"
    #         )
    ollama_ef = OllamaEmbeddingFunction(url="http://localhost:11434/api/embeddings", model_name="llama3:70b")
    collection = client.get_collection(name="address_business_guide_llama3", embedding_function=ollama_ef)

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=10)
    docs = text_splitter.split_documents(content)

    for doc in tqdm(docs):
        uuid_val = uuid.uuid1()
        # print("Inserted documents for ", uuid_val)
        collection.add(ids=[str(uuid_val)], documents=doc.page_content)

In [19]:
insert(pages)

 12%|█▏        | 140/1126 [4:49:02<33:55:41, 123.88s/it]


KeyboardInterrupt: 

* 시간: 5시간 동안 12% 돌아감. 이건 못 쓸듯!

In [ ]:
import chromadb
from langchain.vectorstores import Chroma
# from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings

def queryDB(query):
    client = chromadb.HttpClient(host="localhost", port=8000)
    # embedding_function = OpenAIEmbeddings()
    embedding_function = OllamaEmbeddings(model="llama3:70b")
    db4 = Chroma(client=client, collection_name="address_business_guide_llama3", embedding_function=embedding_function)
    docs = db4.similarity_search_with_score(query) ## 유사도 점수까지 확인: https://wikidocs.net/234094
    # retriever = db4.as_retriever(search_type="mmr") ## mmr 검색
    
    return docs # retriever.get_relevant_documents(query)[0]

In [ ]:
query="사물주소에 대해서 설명해줘"
result = queryDB(query)
result